In [ ]:


import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv


In [ ]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 212.4 MB 50 kB/s 
     |████████████████████████████████| 198 kB 34.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=8a3770fa9080b4ccdac59432d18d45adc8b87554b4ba47e098f7c9e85dd1aeeb
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
import findspark
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>

In [ ]:
from pyspark.sql import SparkSession # required to created a dataframe
spark=SparkSession.builder.appName("Basics").getOrCreate() 

import pyspark.sql.functions

In [ ]:
df = spark.read.csv("/kaggle/input/latest-covid19-india-statewise-data/Latest Covid-19 India Status.csv", 
                    inferSchema=True,
                   header=True)

In [ ]:
from pyspark.sql import functions as F

In [ ]:
renamed_df = df.select([F.col(col).alias(col.replace(' ', '_'))for col in df.columns])

In [ ]:
renamed_df.show()

+--------------------+-----------+------+----------+------+----------------+-------------------+---------------+
|           State/UTs|Total_Cases|Active|Discharged|Deaths|Active_Ratio_(%)|Discharge_Ratio_(%)|Death_Ratio_(%)|
+--------------------+-----------+------+----------+------+----------------+-------------------+---------------+
| Andaman and Nicobar|       7560|    10|      7421|   129|            0.13|              98.16|           1.71|
|      Andhra Pradesh|    2004590| 13677|   1977163| 13750|            0.68|              98.63|           0.69|
|   Arunachal Pradesh|      52409|  1127|     51023|   259|            2.15|              97.36|           0.49|
|               Assam|     585689|  8005|    572084|  5600|            1.37|              97.68|           0.96|
|               Bihar|     725605|   102|    715853|  9650|            0.01|              98.66|           1.33|
|          Chandigarh|      65069|    35|     64222|   812|            0.05|               98.7|

In [ ]:
renamed_df = renamed_df.withColumnRenamed('Active_Ratio_(%)', 'Active_Ratio')
renamed_df = renamed_df.withColumnRenamed('Discharge_Ratio_(%)', 'Discharged_Ratio')
renamed_df = renamed_df.withColumnRenamed('Death_Ratio_(%)', 'Death_Ratio')
renamed_df = renamed_df.withColumnRenamed('State/UTs', 'State_UTs')

In [ ]:
renamed_df.createOrReplaceTempView('Data')

In [ ]:
spark.sql('Select * from Data').show()

+--------------------+-----------+------+----------+------+------------+----------------+-----------+
|           State_UTs|Total_Cases|Active|Discharged|Deaths|Active_Ratio|Discharged_Ratio|Death_Ratio|
+--------------------+-----------+------+----------+------+------------+----------------+-----------+
| Andaman and Nicobar|       7560|    10|      7421|   129|        0.13|           98.16|       1.71|
|      Andhra Pradesh|    2004590| 13677|   1977163| 13750|        0.68|           98.63|       0.69|
|   Arunachal Pradesh|      52409|  1127|     51023|   259|        2.15|           97.36|       0.49|
|               Assam|     585689|  8005|    572084|  5600|        1.37|           97.68|       0.96|
|               Bihar|     725605|   102|    715853|  9650|        0.01|           98.66|       1.33|
|          Chandigarh|      65069|    35|     64222|   812|        0.05|            98.7|       1.25|
|        Chhattisgarh|    1004230|   653|    990022| 13555|        0.07|          

In [ ]:
spark.sql('select count(State_UTs) from Data').show()

+----------------+
|count(State_UTs)|
+----------------+
|              36|
+----------------+



In [ ]:
spark.sql('select * from Data order by Active desc limit 5').show()

+--------------+-----------+------+----------+------+------------+----------------+-----------+
|     State_UTs|Total_Cases|Active|Discharged|Deaths|Active_Ratio|Discharged_Ratio|Death_Ratio|
+--------------+-----------+------+----------+------+------------+----------------+-----------+
|        Kerala|    3851984|159870|   3672357| 19757|        4.15|           95.34|       0.51|
|   Maharashtra|    6432649| 53260|   6243034|136355|        0.83|           97.05|       2.12|
|     Karnataka|    2941026| 19810|   2884032| 37184|        0.67|           98.06|       1.26|
|    Tamil Nadu|    2604074| 18603|   2550710| 34761|        0.71|           97.95|       1.33|
|Andhra Pradesh|    2004590| 13677|   1977163| 13750|        0.68|           98.63|       0.69|
+--------------+-----------+------+----------+------+------------+----------------+-----------+



In [ ]:
spark.sql('select * from Data order by Death_Ratio desc limit 5').show()

+-----------+-----------+------+----------+------+------------+----------------+-----------+
|  State_UTs|Total_Cases|Active|Discharged|Deaths|Active_Ratio|Discharged_Ratio|Death_Ratio|
+-----------+-----------+------+----------+------+------------+----------------+-----------+
|     Punjab|     600342|   423|    583564| 16355|        0.07|           97.21|       2.72|
|Uttarakhand|     342786|   310|    335099|  7377|        0.09|           97.76|       2.15|
|Maharashtra|    6432649| 53260|   6243034|136355|        0.83|           97.05|       2.12|
|   Nagaland|      29669|   888|     28170|   611|        2.99|           94.95|       2.06|
|        Goa|     173357|   932|    169239|  3186|        0.54|           97.62|       1.84|
+-----------+-----------+------+----------+------+------------+----------------+-----------+



In [ ]:
spark.sql('select sum(Total_Cases), sum(Active) from Data').show()

+----------------+-----------+
|sum(Total_Cases)|sum(Active)|
+----------------+-----------+
|        32512366|     322327|
+----------------+-----------+



In [ ]:
spark.sql('select * from Data order by Active asc limit 5').show()

+--------------------+-----------+------+----------+------+------------+----------------+-----------+
|           State_UTs|Total_Cases|Active|Discharged|Deaths|Active_Ratio|Discharged_Ratio|Death_Ratio|
+--------------------+-----------+------+----------+------+------------+----------------+-----------+
|Dadra and Nagar H...|      10659|     4|     10651|     4|        0.04|           99.92|       0.04|
| Andaman and Nicobar|       7560|    10|      7421|   129|        0.13|           98.16|       1.71|
|         Lakshadweep|      10318|    28|     10239|    51|        0.27|           99.23|       0.49|
|          Chandigarh|      65069|    35|     64222|   812|        0.05|            98.7|       1.25|
|              Ladakh|      20500|    55|     20238|   207|        0.27|           98.72|       1.01|
+--------------------+-----------+------+----------+------+------------+----------------+-----------+



In [ ]:
import pandas as pd

In [ ]:
pd_df = renamed_df.toPandas()
pd_df.head()

,State_UTs,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharged_Ratio,Death_Ratio
0,Andaman and Nicobar,7560,10,7421,129,0.13,98.16,1.71
1,Andhra Pradesh,2004590,13677,1977163,13750,0.68,98.63,0.69
2,Arunachal Pradesh,52409,1127,51023,259,2.15,97.36,0.49
3,Assam,585689,8005,572084,5600,1.37,97.68,0.96
4,Bihar,725605,102,715853,9650,0.01,98.66,1.33


In [ ]:
pd_df.describe()

,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharged_Ratio,Death_Ratio
count,3.600000e+01,36.000000,3.600000e+01,36.000000,36.000000,36.000000,36.000000
mean,9.031213e+05,8953.527778,8.820634e+05,12104.388889,1.263611,97.471667,1.265278
std,1.316180e+06,27649.920018,1.276942e+06,23385.359462,2.313456,2.239131,0.563770
min,7.560000e+03,4.000000,7.421000e+03,4.000000,0.010000,87.300000,0.040000
25%,7.194125e+04,158.250000,6.865075e+04,806.750000,0.065000,97.357500,0.955000
50%,4.667220e+05,910.000000,4.572770e+05,5366.000000,0.570000,98.185000,1.295000
75%,1.002800e+06,6373.000000,9.880322e+05,13603.750000,1.065000,98.652500,1.607500
max,6.432649e+06,159870.000000,6.243034e+06,136355.000000,12.330000,99.920000,2.720000


In [ ]:
pd_df.corr()

,Total_Cases,Active,Discharged,Deaths,Active_Ratio,Discharged_Ratio,Death_Ratio
Total_Cases,1.000000,0.657405,0.999924,0.904746,-0.083121,0.062078,0.092816
Active,0.657405,1.000000,0.649104,0.373963,0.217516,-0.181338,-0.173172
Discharged,0.999924,0.649104,1.000000,0.906137,-0.088095,0.066903,0.094055
Deaths,0.904746,0.373963,0.906137,1.000000,-0.125013,0.055103,0.292854
Active_Ratio,-0.083121,0.217516,-0.088095,-0.125013,1.000000,-0.969908,-0.251648
Discharged_Ratio,0.062078,-0.181338,0.066903,0.055103,-0.969908,1.000000,0.008449
Death_Ratio,0.092816,-0.173172,0.094055,0.292854,-0.251648,0.008449,1.000000


In [ ]:
# Make 4 histograms: 'Total_Cases', 'Active', 'Deaths', 'Discharge'
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=2, subplot_titles=['Total_Cases', 'Active', 'Deaths', 'Discharged'])

total_cases = go.Histogram(x=pd_df['Total_Cases'], nbinsx=20, name='Total Cases')
active = go.Histogram(x=pd_df['Active'], nbinsx=20, name='Active')
deaths = go.Histogram(x=pd_df['Deaths'], nbinsx=20, name='Deaths')
discharged = go.Histogram(x=pd_df['Discharged'], nbinsx=20, name='Discharged')

fig.add_traces(total_cases, 1, 1)
fig.add_traces(active, 1, 2)
fig.add_traces(deaths, 2, 1)
fig.add_traces(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Total_Cases', 'Active', 'Deaths', 'Discharged'])

total_cases = go.Box(x=pd_df['Total_Cases'],  name='Total Cases', text= pd_df['State_UTs'])
active = go.Box(x=pd_df['Active'],  name='Active', text= pd_df['State_UTs'])
deaths = go.Box(x=pd_df['Deaths'], name='Deaths', text= pd_df['State_UTs'])
discharged = go.Box(x=pd_df['Discharged'], name='Discharged', text= pd_df['State_UTs'])

fig.add_traces(total_cases, 1, 1)
fig.add_traces(active, 1, 2)
fig.add_traces(deaths, 2, 1)
fig.add_traces(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=['Total_Cases', 'Active', 'Deaths', 'Discharged'])

total_cases = go.Bar(y=pd_df['Total_Cases'],  name='Total Cases', hovertext= pd_df['State_UTs'])
active = go.Bar(y=pd_df['Active'],  name='Active', hovertext= pd_df['State_UTs'])
deaths = go.Bar(y=pd_df['Deaths'], name='Deaths', hovertext= pd_df['State_UTs'])
discharged = go.Bar(y=pd_df['Discharged'], name='Discharged', hovertext= pd_df['State_UTs'])

fig.add_traces(total_cases, 1, 1)
fig.add_traces(active, 1, 2)
fig.add_traces(deaths, 2, 1)
fig.add_traces(discharged, 2, 2)

fig.update_layout(showlegend=False)
fig.show()

In [ ]:
fig = go.Figure([go.Bar(y=pd_df['Total_Cases'],  name='Total Cases', hovertext= pd_df['State_UTs'], x=pd_df['State_UTs']),
                go.Bar(y=pd_df['Discharged'], name='Discharged', hovertext= pd_df['State_UTs'], x=pd_df['State_UTs'])])

fig.update_layout(barmode='group')
fig.update_layout(title='Total & Discharged Cases')
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=['Total_Cases', 'Active', 'Deaths', 'Discharged'],
                    specs=[
                            [{'type':'domain'}, {'type':'domain'}],
                            [{'type':'domain'},{'type':'domain'}]
                          ])


total_cases = go.Pie(values=pd_df['Total_Cases'],  name='Total Cases', labels=pd_df['State_UTs'])
active = go.Pie(values=pd_df['Active'],  name='Active', labels=pd_df['State_UTs'])
deaths = go.Pie(values=pd_df['Deaths'], name='Deaths', labels=pd_df['State_UTs'])
discharged = go.Pie(values=pd_df['Discharged'], name='Discharged', labels=pd_df['State_UTs'])

fig.add_traces(total_cases, 1, 1)
fig.add_traces(active, 1, 2)
fig.add_traces(deaths, 2, 1)
fig.add_traces(discharged, 2, 2)

fig.update_traces(hoverinfo='percent+label')
fig.update_layout(showlegend=False)

fig.update_traces(textposition='inside')

fig = go.Figure(fig)
fig.show()

In [ ]:
fig = go.Figure([go.Scatter(x=pd_df['Total_Cases'], y=pd_df['Deaths'], mode='markers')])
fig.update_layout(title='Deaths vs Total Cases', xaxis_title='Total Cases', yaxis_title='Deaths')
fig.show()

In [ ]:
x = pd_df['Total_Cases'].to_numpy()
y = pd_df['Deaths'].to_numpy()

x.shape, y.shape

((36,), (36,))

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x.reshape((len(x), 1)), y)
(model.intercept_, model.coef_[0])

(-2413.439788501162, 0.01607517067155438)

In [ ]:
model.score(x.reshape((len(x), 1)), y)

0.8185661855516211

In [ ]:
0.904746**2

0.8185653245160001

In [ ]:
predictions = model.predict(x.reshape((len(x), 1)))
predictions.shape

(36,)

In [ ]:
fig = go.Figure([go.Scatter(x=pd_df['Total_Cases'], mode='markers',y=pd_df['Deaths'], name='Actual Deaths'),
                 go.Scatter(x=pd_df['Total_Cases'], y=predictions, name='Predicted Deaths')
               ])
fig.update_layout(title='Deaths vs Total Cases', xaxis_title='Total Cases', yaxis_title='Deaths')
fig.show()